# HW 10
__CS 216, Everything Data, Spring 2020__

__DUE: Monday April 13 by 4:40 pm US Eastern Time (class time)__

This assignment is about working with text data to compare document similarity and model document topics. You will include all of your answers for this assignment within this notebook. You will then convert your notebook to a .pdf and a .py file to submit to gradescope (submission instructions are included at the bottom).

Please take note of the [course collaboration policy](https://sites.duke.edu/compsci216s2020/policies/). You may work alone or with a single partner. If you work with a partner, you may not split up the assignment; you should work together or complete parts independently and come together to discuss your solutions. In either case, you are individually responsible for your work, and should understand everything in your submission.

# Part 1: Tokenizing and Counting Words
An important step for working with text data is converting "raw text" into more machine learning friendly formats. In this part of the homework, you will go through the process of taking multiple short documents written in plain English and converting them into tf-idf scores. The problems will guide you through this process in three parts: tokenizing, counting token frequencies, and finally calculating tf-idf scores. Below, we define a corpus (a collection of documents) of three very short documents describing three different kinds of animals. 

In [1]:
import pandas as pd
import numpy as np
import re # Python library for regular expressions

corpus = ['This is the first document. It is about cats. I like cats, do you? This is it.',
          'This document is the second document. The document is about dogs. They like to play and go on walks. Do you like dogs?',           
          'This is the third document. It is about birds. Birds like to fly. I like to listen to birds.'
         ]

print(corpus[0], '\n')
print(corpus[1], '\n')
print(corpus[2], '\n')

This is the first document. It is about cats. I like cats, do you? This is it. 

This document is the second document. The document is about dogs. They like to play and go on walks. Do you like dogs? 

This is the third document. It is about birds. Birds like to fly. I like to listen to birds. 



### Problem A
Tokenize each of the three documents above. **That is, for each article, create a list of words appearing in that article (including duplicates), and print the resulting lists.** When tokenizing, you should remove blank spaces and punctuation (including the characters '.', ',', and '?'). You should also convert all characters to lowercase. You do *not* need to worry about "stemming" for this problem. As an example, the setence "Fish swim, whereas cats do not." would be tokenized to ['fish', 'swim', 'whereas', 'cats' 'do', 'not']. You may find Python string operations useful: https://docs.python.org/3.7/library/string.html. In particular, you may consider using the lower(), split(), and strip() functions. You are also welcome to use regular expressions, which we discussed earlier in the course. Regular expressions are supported in Python with the re library imported above; you can find more here: https://docs.python.org/3.7/library/re.html. Note that you are not required to use regular expressions, and this problem can be solved only using Python string operations.

In [15]:
def tokenize(text):
    text = re.sub(r"[.,?]", "", text)
    text = text.lower()
    text = text.strip()
    text = text.split(" ")
    return text

print(tokenize(corpus[0]))
print(tokenize(corpus[1]))
print(tokenize(corpus[2]))

['this', 'is', 'the', 'first', 'document', 'it', 'is', 'about', 'cats', 'i', 'like', 'cats', 'do', 'you', 'this', 'is', 'it']
['this', 'document', 'is', 'the', 'second', 'document', 'the', 'document', 'is', 'about', 'dogs', 'they', 'like', 'to', 'play', 'and', 'go', 'on', 'walks', 'do', 'you', 'like', 'dogs']
['this', 'is', 'the', 'third', 'document', 'it', 'is', 'about', 'birds', 'birds', 'like', 'to', 'fly', 'i', 'like', 'to', 'listen', 'to', 'birds']


### Problem B
When analyzing text data, we often model a document by the number of times different words appear. Now that you have tokenized the three documents in our corpus, we should be able to do this counting. **Start by finding the total vocabulary of these three documents: the words that appear in at least one of the documents. Print this vocabulary**, and double check that all of the words from the corpus appear and that none of the words have puntuation or blank spaces in them. 

Then, for each word in the vocabulary and for each document in the corpus, **count how many times each word appears in each of the documents** (you will want to use your tokenized representation of the documents for the counting). **Identify (print or write) the most common word in each document** (you can print any of the most common words if there is a tie).

In [25]:
totVoc = set(tokenize(corpus[0])).union(set(tokenize(corpus[1])).union(set(tokenize(corpus[2]))))
def count(tok):
    counts = dict()
    for word in totVoc:
        counts[word] = 0
    for word in tok:
        counts[word] = counts[word] + 1
    return counts
for i in range(3):
    print(count(tokenize(corpus[i])))


{'fly': 0, 'first': 1, 'to': 0, 'third': 0, 'walks': 0, 'listen': 0, 'cats': 2, 'the': 1, 'and': 0, 'is': 3, 'they': 0, 'you': 1, 'like': 1, 'birds': 0, 'document': 1, 'go': 0, 'it': 2, 'do': 1, 'dogs': 0, 'play': 0, 'i': 1, 'about': 1, 'on': 0, 'this': 2, 'second': 0}
{'fly': 0, 'first': 0, 'to': 1, 'third': 0, 'walks': 1, 'listen': 0, 'cats': 0, 'the': 2, 'and': 1, 'is': 2, 'they': 1, 'you': 1, 'like': 2, 'birds': 0, 'document': 3, 'go': 1, 'it': 0, 'do': 1, 'dogs': 2, 'play': 1, 'i': 0, 'about': 1, 'on': 1, 'this': 1, 'second': 1}
{'fly': 1, 'first': 0, 'to': 3, 'third': 1, 'walks': 0, 'listen': 1, 'cats': 0, 'the': 1, 'and': 0, 'is': 2, 'they': 0, 'you': 0, 'like': 2, 'birds': 3, 'document': 1, 'go': 0, 'it': 1, 'do': 0, 'dogs': 0, 'play': 0, 'i': 1, 'about': 1, 'on': 0, 'this': 1, 'second': 0}


'Is' is the most common word in document 1.
'Document' is the most common word in document 2.
'To' is the most common word in document 3.

### Problem C
Not all of the most common words are informative. Some of them are common just because they are standard verbs or prepositions that appear in many or most setences. To look for words that are particularly important or *salient*, we often want to compute tf-idf (term frequency times inverse document frequency) scores to identify important words that appear frequently in one document but are uncommon across documents.

Formally, the tf-idf score of a given word x in a given document is equal to $$\log_{10}(1+c(x)) \times \log_{10} \left( \frac{N_{docs}}{DF(x)} \right)$$ where c(x) is the count of the number of times that x appears in the document, N_docs is the total number of documents in the corpus, and DF(x) is the number of documents in which x appears at least once. **Compute the tf-idf score for each word in the vocabulary for each document. Identify the highest tf-idf score word for each document.** Hint: These words should be clearly related to the topics of the documents.

In [32]:
import math

tok1 = tokenize(corpus[0])
tok2 = tokenize(corpus[1])
tok3 = tokenize(corpus[2])
count1 = count(tok1)
count2 = count(tok2)
count3 = count(tok3)

dfs = dict()
for word in totVoc:
        dfs[word] = 0
        if count1[word] > 0:
            dfs[word] = dfs[word] + 1
        if count2[word] > 0:
            dfs[word] = dfs[word] + 1
        if count3[word] > 0:
            dfs[word] = dfs[word] + 1
            
def calcScore(tok, count):
    scores = dict()
    for word in totVoc:
        scores[word] = math.log((1 + count[word]), 10) * math.log((3/(dfs[word])), 10)
    return scores
        
print(calcScore(tok1, count1))
print(calcScore(tok2, count2))
print(calcScore(tok3, count3))

{'fly': 0.0, 'first': 0.14362780923945323, 'to': 0.0, 'third': 0.0, 'walks': 0.0, 'listen': 0.0, 'cats': 0.227644691705265, 'the': 0.0, 'and': 0.0, 'is': 0.0, 'they': 0.0, 'you': 0.0530087509499967, 'like': 0.0, 'birds': 0.0, 'document': 0.0, 'go': 0.0, 'it': 0.08401688246581175, 'do': 0.0530087509499967, 'dogs': 0.0, 'play': 0.0, 'i': 0.0530087509499967, 'about': 0.0, 'on': 0.0, 'this': 0.0, 'second': 0.0}
{'fly': 0.0, 'first': 0.0, 'to': 0.0530087509499967, 'third': 0.0, 'walks': 0.14362780923945323, 'listen': 0.0, 'cats': 0.0, 'the': 0.0, 'and': 0.14362780923945323, 'is': 0.0, 'they': 0.14362780923945323, 'you': 0.0530087509499967, 'like': 0.0, 'birds': 0.0, 'document': 0.0, 'go': 0.14362780923945323, 'it': 0.0, 'do': 0.0530087509499967, 'dogs': 0.227644691705265, 'play': 0.14362780923945323, 'i': 0.0, 'about': 0.0, 'on': 0.14362780923945323, 'this': 0.0, 'second': 0.14362780923945323}
{'fly': 0.14362780923945323, 'first': 0.0, 'to': 0.1060175018999934, 'third': 0.14362780923945323,

'cats' has the highest score for the first document.
'dogs' has the highest score for the second document.
'birds' has the highest score for the third document.

# Part 2: Text Data Analysis with Scikit-Learn
Now that you understand the basics of tokenizing and computing tf-idf scores for words, the second part of this homework will explore using scikit-learn to perform topic modeling of real world news articles. Below, we import a selection of 500 short news articles. Each is tagged with one of five possible topics (business, sport, entertainment, politics, or tech). Our goal will be to use a tf-idf word vector representation of the articles to identify articles about similar topics.

In [33]:
articles = pd.read_csv('bbc.csv')
articles.head()

,category,text
0,business,go-ahead for balkan oil pipeline albania bulg...
1,business,us interest rate rise expected us interest rat...
2,sport,gardener battles to narrow win jason gardener ...
3,entertainment,us afflicted with awards fatigue the film wo...
4,politics,tory backing for id cards the tories are to ba...


To start, we need to tokenize all of the articles, count tokens, and compute tf-idf scores for all of the words and articles. In Part 1 you went through this process yourself; in this part of the homework we introduce the scikit-learn vectorizer for completing this task. Recall from HW 6 that scikit-learn is the standard open source library for machine learning in Python; the tf-idf vectorizer is a useful preprocessing step for many machine learning tasks workign with text data: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html. It is called a "vectorizer" because it represents each document as a vector, where a given dimension of the vector corresponds to the tf-idf score of a particular word in the vocabulary for that document. We will represent documents as these tf-idf vectors in order to compare them to one another. 

Note that we set a minimum and maximum document frequency; this means that we are only including words from the vocabulary that appear in at least 1% of the documents and at most 20% of the documents. The specific numbers are arbitrary, but in general this step is filtering to focus on words that appear in enough documents to be useful for making comparisons, but not so frequently that they are likely to be, say, standard English prepositions. With these parameters we filter the vocabulary down to 3475 words of interest.

In [34]:
# Run this code to tokenize and compute tf-idf scores.
# You should not need to modify this code.
from sklearn.feature_extraction.text import TfidfVectorizer

# Do not modify these parameters 
vectorizer = TfidfVectorizer(min_df=0.01, max_df=0.2,  norm=None)
tfidf_scores = vectorizer.fit_transform(articles['text'])

An important observation about the results from a large collection of documents (we have 500 articles, not all of which are related to each other in topic or vocabulary) is that many words (and especially salient words) won't appear in many of the articles. In other words, most of the entries in most of the tf-idf vectors are 0. For that reason, scikit-learn stores the results of the tf-idf vectorizer in a *sparse* format, only storing the nonzero entries. `tfidf_scores` from the above code cell is in this format, about which you can read more at https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix. We can recover the full matrix (one row for each article, and one column for each word in the vocabulary) using the toarray() function, but most of the entries will be 0. We do just this below for purpose of visualizing the results in a table.

In [35]:
# Run this code to visualize the "dense" version
# of the tf-idf matrix. Each row corresponds to an 
# article, and each column corresponds to a "word" 
# from the vocabulary.
tfidf_matrix = tfidf_scores.toarray()

df_tfidf_scores = pd.DataFrame(tfidf_matrix, columns=vectorizer.get_feature_names())
df_tfidf_scores.head()

,000m,10,100,100m,11,110,12,120,13,14,...,yen,yet,york,young,younger,youngsters,your,youth,yukos,zealand
0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
2,5.137165,0.0,0.0,4.818711,6.609166,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.000000,3.304583,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,4.914021,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.478936,0.000000,0.0,0.0,0.0,0.0,0.0


Now we want to use these tf-idf scores to compare articles to one another. Each article is represented as its tf-idf score vector (that is, article `i` is represented by the vector corresponding to row `i` in the above table). For every pair of articles, we will compute the cosine similarity between these tf-idf score vectors. Recall that the cosine similarity of two d-dimensional vectors x and y is $$\frac{x \cdot y}{\|x\|_2 \|y\|_2} = \frac{\sum_{i=1}^{d} x_i y_i}{\left(\sqrt{\sum_{i=1}^{d} x_i^2}\right) \left(\sqrt{\sum_{i=1}^{d} y_i^2}\right)}.$$ Scikit-learn implements computing all of these pairwise similarities in the pairwise metrics library: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html. 

In [41]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(tfidf_scores)

### Problem D
Now let's use these cosine similarity scores. **Identify the five articles most similar to article 0** (where by article 0 we mean the article corresponding to row 0 in the original dataset). More concretely, identify the five articles (by the index of the rows in which they appear in the original dataset) with the greatest cosine similarity to article 0 using the tf_idf vectors. **Write the rows of these five articles. Also, print the text of article 0 and one or more of the articles you found.**

In [64]:
myList = cos_sim[0][1:].tolist()
while len(myList) > 5:
    myList.remove(min(myList))
myList2 = cos_sim[0][1:].tolist()
indexes = []
for num in myList:
    indexes.append(myList2.index(num))
print("rows: ", indexes)
print("text from article 0:")
print(articles["text"][0])
print("text from article 94:")
print(articles["text"][94])

rows:  [94, 249, 354, 378, 492]
text from article 0:
go-ahead for balkan oil pipeline albania  bulgaria and macedonia has given the go ahead for the construction of a $1.2bn oil pipeline that will pass through the balkan peninsula.  the project aims to allow alternative ports for the shipping of russian and caspian oil  that normally goes through turkish ports. it aims to transport 750 000 daily barrels of oil. the pipeline will be built by the us-registered albanian macedonian bulgarian oil corporation (ambo). the 912km pipeline will run from the bulgarian port of burgas  over the black sea to the albanian city of vlore on the adriatic coast  crossing macedonia.  the project was conceived in 1994 but it was delayed because of the lack of political support. by signing the agreement on tuesday  the prime ministers of bulgaria  albania and macedonia have overcome the problem.  this is one of the most important infrastructure projects for regional  eu  and euro-atlantic integration for th

### Problem E
Now that we know we can use cosine similarity scores to find similar articles, we can use that information to model the topics (business, sport, entertainment, politics, or tech) of the articles in our dataset. Below, we split the data into training and testing datasets as in other prediction problems (from HW 6, for example). In particular, we split the tfidf scores, so each row in the resulting `X_train` and `X_test` numpy arrays correspond to the tfidf score vector of a particular article. `y_train` and `y_test` values are the true topics of the corresponding articles, taken from the `category` column of the original dataset. **Your task is to make predictions for the topics of the articles in the test dataset (without using `y_test`, of course). Then, compute the accuracy of your predictions (the scikit-learn code for this is included below). For full credit, your accuracy should be at least 0.8.**

To get started, we suggest you make your predictions based on the k-nearest neighbor algorithm that we discussed earlier in the course. You are welcome to use the scikit-learn implementation (imported below), but it does not implement cosine similarity directly. Therefore, if you want to use scikit-learn, you will need to precompute distances based on cosine similarity (you can use the `cosine_similarity` function from above). To use precomputed distances, you can set `metric='precomputed` when initializing the classifier, and then the fit and predict methods will require you to pass distance matrices rather than feature matrices. The fit method will expect a square matrix of distances between every pair of articles in the training data, and the predict method will expect a 2d numpy array with a row for every test article and a column for every train article, where the corresponding entry gives the distance between that train and test example. You can look up the full documentation at https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier. We recommend using `1.001 - cosine similarity(u, v)` as the 'distance' between two tfidf vectors u and v based on cosine similarity. 

If you prefer, you can also make the predictions yourself without using scikit-learn. To do so, for each test article, simply find the k train articles with maximum cosine similarity and predict the most common topic of those train articles. Even using k=1 (so just finding the single most similar train article and predicting its topic) should be sufficient to get an accuracy of 0.8.

In [66]:
# Import libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Get the topics and split the data into train and test sets
# Do not change this code
topics = articles['category'].values
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, topics, test_size=0.3, random_state=0)

In [69]:
# Write your code for problem E to make predictions here
cos_sim_train = cosine_similarity(X_train)
cos_sim_test = cosine_similarity(X_test)
neigh = KNeighborsClassifier(metric = 'precomputed')


[[1.         0.02194798 0.02334163 ... 0.02220758 0.06968885 0.06144986]
 [0.02194798 1.         0.02958468 ... 0.05782693 0.03464264 0.0165196 ]
 [0.02334163 0.02958468 1.         ... 0.06932801 0.01356733 0.0156424 ]
 ...
 [0.02220758 0.05782693 0.06932801 ... 1.         0.03362949 0.01462828]
 [0.06968885 0.03464264 0.01356733 ... 0.03362949 1.         0.07006504]
 [0.06144986 0.0165196  0.0156424  ... 0.01462828 0.07006504 1.        ]]


In [ ]:
# Prints accuracy of predictions
# Replace 'pred' with your predictions
print("Accuracy of topic model: ", accuracy_score(pred, y_test))

## Submitting HW 10
1. Double check that you have written all of your answers along with your supporting work in this notebook. Make sure you save the complete notebook.
1. Double check that your entire notebook runs correctly and generates the expected output. To do so, you can simply select Kernel -> Restart and Run All. 
2. You will download two versions of your notebook to submit, a .pdf and a .py. To create a PDF, we reccomend that you select File --> Download as --> HTML (.html). Open the downloaded .html file; it should open in your web broser. Double check that it looks like your notebook, then print a .pdf using your web browser (you should be able to select to print to a pdf on most major web browsers and operating systems). Check your .pdf for readability: If some long cells are being cut off, go back to your notebook and split them into multiple smaller cells. To get the .py file from your notebook, simply select File -> Download as -> Python (.py) (note, we recognize that you may not have written any Python code for this assignment, but will continue the usual workflow for consistency). 
3. Upload the .pdf to gradescope under hw 10 report and the .py to gradescope under hw 10 code. If you work with a partner, only submit one document for both of you, but be sure to add your partner using the [group feature on gradescope](https://www.gradescope.com/help#help-center-item-student-group-members).